# Capstone Project - The Battle of the Neighborhoods in Warsaw
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)




## Introduction: Business Problem <a name="introduction"></a>

Warsaw is the capital of Poland and the largest city in the country. The city's population is 1.8 million with a density of 3,469 people per square kilometer. Warsaw is divided into 18 districts with different population densities and areas. As a person who lived there for 5 years, I decided to choose this place for my project.

There are numerous companies and universities in the city attracting a huge number of interested people. Everyone is interested in various attractions in the city, depending on their needs. A family with children will have different needs than a student. The need to find an interesting and cheap point for yourself is a very important aspect. Additional needs are the knowledge of attractions available in the selected area.

Data on average housing prices will be presented on the map for individual districts. Additionally, we will have clustered information about venue density.

## Data <a name="data"></a>

Based on the definition of our problem, the factors that will influence our decision are:
* number of points of interest broken down by type;
* Average prices of apartments in the area.

The following sources will be used to obtain the necessary information:

* Wikipedia - Administrative divisions of Warsaw;
* Forsquare API to obtain information about the points of interest;
* Public data on average prices per square meter in individual districts.